# 作成した教材をGitHubへバックアップします
<HR>

ホームディレクトリで作成した教材をGitHubリポジトリへバックアップします。

### 1. バックアップ用GitHubリポジトリの準備

* 既にバックアップ用GitHubリポジトリが作成済みの場合は、この項を読み飛ばして結構です。
* 初めて教材をGitHubリポジトリへバックアップする場合は、[GitHubリポジトリの準備](03_PrepareGitHub.ipynb) Notebookの「1. GitHubで新しいリポジトリを作成
」にしたがって、バックアップ用のGitHubリポジトリを作成して下さい。

### 2. GitHubリポジトリへの接続情報を設定します

* バックアップ先のGitHubリポジトリへの接続情報を設定します。
* [GitHubリポジトリの準備](03_PrepareGitHub.ipynb) Notebookの「2. GitHubリポジトリ接続情報の設定」を使って、バックアップ用のGitHubリポジトリの接続情報を設定して下さい。

### 3. バックアップの準備

このNotebookを実行すると、ホームディレクトリ直下の下記のフォルダをGitHubリポジトリに保存します。<br/>
**バックアップしたいファイル/フォルダは、下記のフォルダに移動してください。**
* textbook
  + 教材Notebookおよび教材Notebookから呼び出すスクリプトなど
* draft
  + 教材以外で保存して置きたいNotebook、その他ファイル

注） GitHubには「test」フォルダも保存されます。
* 「test」フォルダは、この教材の自動テストに必要なファイルを保存するフォルダです。（初期状態で、テスト手順Notebookのテンプレートが配置されています）
* **「test」フォルダにはその他のファイルを格納しないで下さい。**

### 4. その他にバックアップしたいファイル/フォルダがある場合

* もし、textbook, draft以外に保存したいファイルまたはフォルダがある場合は、INCLUDES にスペース区切りで列挙して下さい。
  + textboo, draft, testフォルダは必ずバックアップに保存されます。
  + 管理用Notebookフォルダ「admin_tools」はバックアップに保存されません
```
INCLUDES="work"
```

**次のセルでバックアップに含めたいファイル/フォルダを指定して、セルを実行して下さい。**

In [ ]:
INCLUDES=""

### 5. GitHubにpushします

* 指定したGitHub repositoryに保存します。

**次のセルで、commitメッセージを設定して下さい。**

In [ ]:
COMMIT_MESSAGE=""

**次のセルを実行して、GitHubに保存して下さい。**

In [ ]:
# check setting
do_backup=1
[ -z "$GITHUB_REPO" ]  && {
    echo "GITHUB_REPO is not set."
    do_backup=0
}
if [ "$do_backup" -eq 1 ] && [ -z $COMMIT_MESSAGE ]; then
    echo "COMMIT_MESSAGE is not set."
    do_backup=0
fi

if [ "$do_backup" -eq 1 ]; then
    # generate .gitignore
    cd $HOME
    INCLUDES="textbook draft test "$INCLUDES
    includes_reg=""
    for item in $INCLUDES; do
        includes_reg=$includes_reg"|$item"
    done
    SKIPS_REG="\.$|\.\.$"$includes_reg
     : > .gitignore
    for item in $(ls -a); do
        [[ "$item" =~ $SKIPS_REG ]] && continue
        echo $item >> .gitignore
    done

    # forcibly exclude admin-tools
    [[ "$INCLUDES" =~ admin-tools ]] && {
        echo "admin-tools" >> .gitignore  
        replace_includes=""
        for item in $INCLUDES; do
            [[ "$item" =~ admin-tools ]] && continue
            replace_includes=$replace_includes"$item "
        done
        INCLUDES=$replace_includes
    }
    echo "INCLUDES: "$INCLUDES

    # push to GitHub　repository
    git add $INCLUDES
    git commit -m "$COMMIT_MESSAGE"
    git push origin master
fi

echo
echo "... backup finished."